In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd

In [2]:
# Top best seller
root_url = 'https://www.audible.com/adblbestsellers?ref_pageloadid=zczZf55x7irPQP6H&ref=a_ep_audiob_t1_navTop_pl0cg1c0r0&pf_rd_p=371ebad3-b9dd-4d26-9542-89be13712099&pf_rd_r=GK9A8SEXM19V3AWX31BX&pageLoadId=LXiG5fyDg2ousK0L&creativeId=711b5140-9c53-4812-acee-f4c553eb51fe'

chrome_driver_path = r'D:\Project\Python\chrome-driver\chromedriver-win64\chromedriver.exe'
web_service = Service(executable_path=chrome_driver_path)

# Configure web driver
web_option = Options()
web_option.add_argument("--headless")  # hide GUI
web_option.add_argument("--window-size=1920,1080")  # set window size to native GUI size
web_option.add_argument("start-maximized")  # ensure window is full-screen

web_driver = webdriver.Chrome(service=web_service, options=web_option)
web_driver.get(url=root_url)

In [3]:
paging_item = web_driver.find_elements(by=By.XPATH, value='//ul[contains(@class, "pagingElements")]//li')

page_numbers = int(paging_item[-2].text)
print('Number of pages: ', page_numbers)

books_title = []
books_author = []
books_length = []

# for in range of pages
for page in range(1, page_numbers + 1):
    print("Scraping page: ", page)

    box_items = WebDriverWait(driver=web_driver, timeout=10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="adbl-impression-container "]')))
    books_item = WebDriverWait(driver=box_items, timeout=10).until(EC.presence_of_all_elements_located((By.XPATH, './/li[contains(@class, "productListItem")]')))

    for book in books_item:
        book_title = book.find_element(By.XPATH, './/h3[contains(@class, "bc-heading")]').text
        book_title = re.sub('\d+\.\s', '', book_title)
        books_title.append(book_title)
        books_author.append(book.find_element(By.XPATH, './/li[contains(@class, "authorLabel")]').text.replace('By: ', ""))
        books_length.append(book.find_element(By.XPATH, './/li[contains(@class, "runtimeLabel")]').text.replace('Length: ', ''))

    # Click to next page
    next_item = web_driver.find_element(By.XPATH, '//span[contains(@class, "nextButton")]')
    next_item.click()

books_df = pd.DataFrame({'title': books_title, 'author': books_author, 'length': books_length})
books_df.to_csv('book_data.csv')

Number of pages:  5
Scraping page:  1
Scraping page:  2
Scraping page:  3
Scraping page:  4
Scraping page:  5


In [4]:
web_driver.quit()